In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from google.colab import drive
drive.mount('/content/drive')

# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'Valence_Value', 'frameTime'], axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define the number of folds for cross-validation
num_folds = 2

# Initialize lists to store evaluation results
mse_a = []
mae_a = []
rmse_a = []
r2_a = []
f1_a = []
accuracy_a = []
precision_a = []
recall_a = []
train_mse_a = []
test_mse_a = []
train_accuracy_a = []
test_accuracy_a = []

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_reshaped):
    X_train, X_val = X_reshaped[train_index], X_reshaped[test_index]
    y_a_train, y_a_val = y_a[train_index], y_a[test_index]

    # Split the validation set further into val and test sets
    val_size = len(X_val) // 2
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_a_val, y_a_test = y_a_val[:val_size], y_a_val[val_size:]

    # Build the BiLSTM model for Arousal
    model_a = Sequential()
    model_a.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_a.add(Flatten())

    # Add DNN layers with dropout
    dnn_layers = [512, 256, 128, 64]  # Modify this list as needed

    for layer_size in dnn_layers:
        model_a.add(Dense(layer_size, activation='relu'))

    model_a.add(Dense(1))
    model_a.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

    # Train the Arousal model
    model_a.fit(X_train, y_a_train, validation_data=(X_val, y_a_val), epochs=65, batch_size=32)
    threshold = 0
    # Evaluate the model on the training set
    train_predictions_a = model_a.predict(X_train)
    train_mse = mean_squared_error(y_a_train, train_predictions_a)
    train_accuracy = accuracy_score(y_a_train >= threshold, train_predictions_a >= threshold)
    train_mse_a.append(train_mse)
    train_accuracy_a.append(train_accuracy)

    # Evaluate the model on the test set
    test_predictions_a = model_a.predict(X_test)
    test_mse = mean_squared_error(y_a_test, test_predictions_a)
    test_accuracy = accuracy_score(y_a_test >= threshold, test_predictions_a >= threshold)
    test_mse_a.append(test_mse)
    test_accuracy_a.append(test_accuracy)

    # Calculate additional evaluation metrics
    mae_a.append(mean_absolute_error(y_a_test, test_predictions_a))
    rmse_a.append(np.sqrt(test_mse))
    r2_a.append(r2_score(y_a_test, test_predictions_a))

    # Convert regression predictions to binary labels

    binary_predictions_a = (test_predictions_a >= threshold).astype(int)
    y_a_test_binary = (y_a_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_a.append(f1_score(y_a_test_binary, binary_predictions_a))
    accuracy_a.append(accuracy_score(y_a_test_binary, binary_predictions_a))
    precision_a.append(precision_score(y_a_test_binary, binary_predictions_a))
    recall_a.append(recall_score(y_a_test_binary, binary_predictions_a))

# Calculate average performance across all folds
average_mse_a = np.mean(test_mse_a)
average_mae_a = np.mean(mae_a)
average_rmse_a = np.mean(rmse_a)
average_r2_a = np.mean(r2_a)
average_f1_a = np.mean(f1_a)
average_accuracy_a = np.mean(accuracy_a)
average_precision_a = np.mean(precision_a)
average_recall_a = np.mean(recall_a)

# Calculate average training and test MSE and accuracy across all folds
average_train_mse_a = np.mean(train_mse_a)
average_test_mse_a = np.mean(test_mse_a)
average_train_accuracy_a = np.mean(train_accuracy_a)
average_test_accuracy_a = np.mean(test_accuracy_a)

print(f'Average Arousal MSE: {average_mse_a:.4f}')
print(f'Average Arousal MAE: {average_mae_a:.4f}')
print(f'Average Arousal RMSE: {average_rmse_a:.4f}')
print(f'Average Arousal R2 Score: {average_r2_a:.4f}')
print(f'Average Arousal F1 Score: {average_f1_a:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_a:.4f}')
print(f'Average Arousal Precision: {average_precision_a:.4f}')
print(f'Average Arousal Recall: {average_recall_a:.4f}')

print("\n\n")

# Print average training and test MSE and accuracy
print(f'Average Train MSE: {average_train_mse_a:.4f}')
print(f'Average Test MSE: {average_test_mse_a:.4f}')
print(f'Average Train Accuracy: {average_train_accuracy_a:.4f}')
print(f'Average Test Accuracy: {average_test_accuracy_a:.4f}')


Mounted at /content/drive
Epoch 1/65
1659/1659 [==============================] - 52s 26ms/step - loss: 0.0478 - val_loss: 0.0477
Epoch 2/65
1659/1659 [==============================] - 42s 25ms/step - loss: 0.0408 - val_loss: 0.0470
Epoch 3/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0389 - val_loss: 0.0440
Epoch 4/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0372 - val_loss: 0.0477
Epoch 5/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0364 - val_loss: 0.0421
Epoch 6/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0356 - val_loss: 0.0441
Epoch 7/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0347 - val_loss: 0.0412
Epoch 8/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0341 - val_loss: 0.0397
Epoch 9/65
1659/1659 [==============================] - 41s 25ms/step - loss: 0.0335 - val_loss: 0.0393
Epoch 10/65
1659/1659 [===============